In [1]:
import pandas as pd
import numpy as np
import sys
sys.path.append('/home/ykkim/Baseline_Analysis/8. AutoML/' )  #모듈 저장 위치 지정 
print(sys.path)
import TPOT_module as TPOT

['/home/ykkim/.jupyter', '/home/ykkim/anaconda3/envs/TPOT/lib/python39.zip', '/home/ykkim/anaconda3/envs/TPOT/lib/python3.9', '/home/ykkim/anaconda3/envs/TPOT/lib/python3.9/lib-dynload', '', '/home/ykkim/anaconda3/envs/TPOT/lib/python3.9/site-packages', '/home/ykkim/Baseline_Analysis/8. AutoML/']


/home/ykkim/anaconda3/envs/TPOT/lib/python3.9/site-packages/tpot/builtins/__init__.py:36: UserWarning: Warning: optional dependency `torch` is not available. - skipping import of NN models.
  warnings.warn("Warning: optional dependency `torch` is not available. - skipping import of NN models.")


In [2]:
#data
data=pd.read_csv('/home/ykkim/Train_data/HS_DataMart_v1.0.csv',encoding='cp949')
data=data.drop(['Unnamed: 0'], axis=1)
data['datetime']=pd.to_datetime(data['datetime'])
data

,datetime,초지_전력사용량,Press_전력사용량,Constant_전력사용량,Wire_전력사용량,AIR_전력사용량,Size_전력사용량,Calender_전력사용량,Drive_전력사용량,Pre-Dryer_전력사용량,...,TURNING ROLL 보류율,사이로 백수 온도,TURNING ROLL_WIRE 진공도 토탈,TURNING ROLL 속도,CENTER NO2 ROLL 속도,1군 DRYER 속도,NO8 DRYER 속도,CALENDER BTM ROLL속도,PM23_평량,PM23_릴속도
0,2022-04-26 00:00:00,1757.6,670.0,185.8,368.2,170.0,1.4,20.4,220.0,111.8,...,86.5844,42.3743,1724.12,1101.2,1101.1,1150.0,1153.3,1152.0,36.8835,NaN
1,2022-04-26 00:15:00,1854.9,670.0,175.8,367.2,160.0,2.5,43.7,210.0,204.7,...,86.6723,42.3743,1702.31,1101.3,1101.4,1150.0,1153.3,1152.1,37.8980,NaN
2,2022-04-26 00:30:00,1620.7,660.0,174.9,369.2,170.0,0.0,8.6,220.0,18.0,...,86.7994,42.5605,1703.96,1101.4,1101.1,1150.0,1153.3,1152.1,37.3890,NaN
3,2022-04-26 00:45:00,1717.2,670.0,175.9,368.2,170.0,1.4,0.0,210.0,111.7,...,86.7853,42.4994,1703.50,1100.8,1101.0,1150.1,1153.3,1152.1,37.8443,NaN
4,2022-04-26 01:00:00,1852.1,670.0,184.8,378.2,170.0,2.7,2.7,220.0,202.7,...,86.7961,42.4994,1722.79,1110.3,1110.3,1160.0,1163.5,1162.1,37.3563,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4361,2022-06-10 10:15:00,1879.7,760.0,235.9,417.3,170.0,0.0,19.5,260.0,17.0,...,80.2992,43.1220,1662.12,1265.0,1265.8,1320.1,1325.9,1324.3,48.8740,1324.1
4362,2022-06-10 10:30:00,1965.7,750.0,234.8,419.3,160.0,1.7,16.0,260.0,113.9,...,81.1845,43.2471,1652.04,1265.0,1263.9,1320.0,1325.9,1324.3,48.1874,1324.1
4363,2022-06-10 10:45:00,2073.0,750.0,236.7,418.4,160.0,3.5,18.4,260.0,207.0,...,81.1510,43.3112,1635.33,1262.3,1261.9,1316.6,1322.0,1320.6,48.7256,1324.1
4364,2022-06-10 11:00:00,1948.2,740.0,235.7,419.3,170.0,1.7,16.0,240.0,115.5,...,85.0735,43.6225,1773.39,1245.8,1246.4,1292.0,1297.3,1296.0,49.0044,1320.3


In [3]:
train_df,test_df=TPOT.train_test_split_df(data, 0.20)
print('Train data :',train_df['datetime'].min(),' ~ ',train_df['datetime'].max(),'(',len(train_df),')')
print('Test data :',test_df['datetime'].min(),' ~ ',test_df['datetime'].max(),'(',len(test_df),')')

Train data : 2022-04-26 00:00:00  ~  2022-06-01 09:00:00 ( 3493 )
Test data : 2022-06-01 09:15:00  ~  2022-06-10 11:15:00 ( 873 )


이상치 제거

In [4]:
train_df['(비정상)지절시 운전']=train_df['(비정상)지절시 운전'].astype(str)
train_df['(비정상)카렌더 SHEET BREAK SENSOR TAG']=train_df['(비정상)카렌더 SHEET BREAK SENSOR TAG'].astype(str)
train_df['(비정상)PICK-UP VACCUM 파괴 밸브 TAG']=train_df['(비정상)PICK-UP VACCUM 파괴 밸브 TAG'].astype(str)

train_df=train_df[train_df['(비정상)지절시 운전'] == '0.0']   
train_df=train_df[train_df['(비정상)카렌더 SHEET BREAK SENSOR TAG'] == '0.0']
train_df=train_df[train_df['(비정상)PICK-UP VACCUM 파괴 밸브 TAG'] == '1.0']
train_df.info()
print('Train data :',train_df['datetime'].min(),' ~ ',train_df['datetime'].max(),'(',len(train_df),')')

#===================================================================================================
test_df['(비정상)지절시 운전']=test_df['(비정상)지절시 운전'].astype(str)
test_df['(비정상)카렌더 SHEET BREAK SENSOR TAG']=test_df['(비정상)카렌더 SHEET BREAK SENSOR TAG'].astype(str)
test_df['(비정상)PICK-UP VACCUM 파괴 밸브 TAG']=test_df['(비정상)PICK-UP VACCUM 파괴 밸브 TAG'].astype(str)

test_df=test_df[test_df['(비정상)지절시 운전'] == '0.0']   
test_df=test_df[test_df['(비정상)카렌더 SHEET BREAK SENSOR TAG'] == '0.0']
test_df=test_df[test_df['(비정상)PICK-UP VACCUM 파괴 밸브 TAG'] == '1.0']
test_df.info()
print('Test data :',test_df['datetime'].min(),' ~ ',test_df['datetime'].max(),'(',len(test_df),')')

/tmp/ipykernel_9642/2787675754.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['(비정상)지절시 운전']=train_df['(비정상)지절시 운전'].astype(str)
/tmp/ipykernel_9642/2787675754.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['(비정상)카렌더 SHEET BREAK SENSOR TAG']=train_df['(비정상)카렌더 SHEET BREAK SENSOR TAG'].astype(str)
/tmp/ipykernel_9642/2787675754.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead



<class 'pandas.core.frame.DataFrame'>
Int64Index: 3349 entries, 0 to 3492
Data columns (total 24 columns):
 #   Column                           Non-Null Count  Dtype         
---  ------                           --------------  -----         
 0   datetime                         3349 non-null   datetime64[ns]
 1   초지_전력사용량                         3349 non-null   float64       
 2   Press_전력사용량                      3349 non-null   float64       
 3   Constant_전력사용량                   3349 non-null   float64       
 4   Wire_전력사용량                       3349 non-null   float64       
 5   AIR_전력사용량                        3349 non-null   float64       
 6   Size_전력사용량                       3349 non-null   float64       
 7   Calender_전력사용량                   3349 non-null   float64       
 8   Drive_전력사용량                      3349 non-null   float64       
 9   Pre-Dryer_전력사용량                  3349 non-null   float64       
 10  After-Dryer_전력사용량                3349 non-null   float64    

#feature engineering
1. (비정상) column downtime 제거  (209개 제거됨)   
2. datetime을 feature(mon,day,hour,min,weekday)화    

 *  feature :    
    1. 'TURNING ROLL 보류율',    
    2. '사이로 백수 온도',
    3. 'TURNING ROLL_WIRE 진공도 토탈',     
    4. 'TURNING ROLL 속도',    
    5. 'CENTER NO2 ROLL 속도',     
    6. '1군 DRYER 속도',   
    7. 'NO8 DRYER 속도',    
    8. 'CALENDER BTM ROLL속도',    
    9. 'PM23_평량',    
    10. 'PM23_릴속도'


In [5]:
train_features=train_df[['datetime',
'TURNING ROLL 보류율',
'사이로 백수 온도',
'TURNING ROLL_WIRE 진공도 토탈',
'TURNING ROLL 속도',
 'CENTER NO2 ROLL 속도',
 '1군 DRYER 속도',
 'NO8 DRYER 속도',
 'CALENDER BTM ROLL속도',
 'PM23_평량',
 'PM23_릴속도']]
train_target=train_df['초지_전력사용량']

test_features=test_df[['datetime', 
'TURNING ROLL 보류율',
'사이로 백수 온도',
'TURNING ROLL_WIRE 진공도 토탈',
'TURNING ROLL 속도',
 'CENTER NO2 ROLL 속도',
 '1군 DRYER 속도',
 'NO8 DRYER 속도',
 'CALENDER BTM ROLL속도',
 'PM23_평량',
 'PM23_릴속도']]
test_target=test_df['초지_전력사용량']

In [6]:
#2. datetime을 feature(mon,day,hour,min,weekday)화  
train_features['mon']=train_features['datetime'].dt.month
train_features['day']=train_features['datetime'].dt.day
train_features['hour']=train_features['datetime'].dt.hour
train_features['min']=train_features['datetime'].dt.minute
train_features['weekday']=train_features['datetime'].dt.weekday
train_features=train_features.drop(['datetime'], axis=1)


#2. datetime을 feature(mon,day,hour,min,weekday)화  
test_features['mon']=test_features['datetime'].dt.month
test_features['day']=test_features['datetime'].dt.day
test_features['hour']=test_features['datetime'].dt.hour
test_features['min']=test_features['datetime'].dt.minute
test_features['weekday']=test_features['datetime'].dt.weekday
testtest=test_features   #추후 시각화용 (use datetime)
test_features=test_features.drop(['datetime'], axis=1)

/tmp/ipykernel_9642/1082184899.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_features['mon']=train_features['datetime'].dt.month
/tmp/ipykernel_9642/1082184899.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_features['day']=train_features['datetime'].dt.day
/tmp/ipykernel_9642/1082184899.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://

pipeline

In [7]:
TPOT.searching_pipeline(train_features,train_target,test_features,test_target,exportfileroot='/home/ykkim/TPOTpipeline/HS_datamart_try1.py')

Imputing missing values in feature set
Optimization Progress:   0%|          | 0/10100 [00:00<?, ?pipeline/s]

/home/ykkim/anaconda3/envs/TPOT/lib/python3.9/site-packages/tpot/builtins/__init__.py:36: UserWarning: Warning: optional dependency `torch` is not available. - skipping import of NN models.
  warnings.warn("Warning: optional dependency `torch` is not available. - skipping import of NN models.")
/home/ykkim/anaconda3/envs/TPOT/lib/python3.9/site-packages/tpot/builtins/__init__.py:36: UserWarning: Warning: optional dependency `torch` is not available. - skipping import of NN models.
  warnings.warn("Warning: optional dependency `torch` is not available. - skipping import of NN models.")


                                                                                   
Generation 1 - Current best internal CV score: -61.25620116808925
                                                                                   
Generation 2 - Current best internal CV score: -60.83652525981388
                                                                                  
Generation 3 - Current best internal CV score: -60.83652525981388
                                                                                  
Generation 4 - Current best internal CV score: -59.96078596119652
                                                                                  
Generation 5 - Current best internal CV score: -59.96078596119652
                                                                                  
Generation 6 - Current best internal CV score: -59.96078596119652
                                                                                  
Generation 7 - Curren